In [1]:
import requests, json, time

In [2]:
from urllib.parse import urlparse, parse_qs

In [3]:
from datetime import datetime, timedelta

In [4]:
client_id = "811981652591-ev7lpah32cjfa45jk4r0m7c8upaeh4q2.apps.googleusercontent.com"
client_secret = "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"
redirect_uri = "https://www.lmu.de/"
authorization_url = "https://accounts.google.com/o/oauth2/auth"
token_url = "https://accounts.google.com/o/oauth2/token"
scope = "https://www.googleapis.com/auth/fitness.sleep.read"

In [5]:
def get_token():
    # Redirect the user to the authorization URL
    auth_params = {
        "client_id": client_id,
        "redirect_uri": redirect_uri,
        "scope": scope,
        "response_type": "code"
    }
    auth_url = f"{authorization_url}?client_id={auth_params['client_id']}&redirect_uri={auth_params['redirect_uri']}&scope={auth_params['scope']}&response_type={auth_params['response_type']}"
    print(f"Please visit the following URL to authorize your application:\n{auth_url}")

    # Handle the authorization code from the callback URL
    authorization_code = parse_url()

    # Exchange the authorization code for an access token
    token_params = {
        "code": authorization_code,
        "client_id": client_id,
        "client_secret": client_secret,
        "redirect_uri": redirect_uri,
        "grant_type": "authorization_code"
    }
    response = requests.post(token_url, data=token_params)
    token_data = response.json()

    # Extract the access token from the response
    access_token = token_data.get("access_token")

    # Use the access token for API requests
    if access_token:
        headers = {"Authorization": f"Bearer {access_token}"}
        # Make API requests using the access token
        # Example: response = requests.get(api_endpoint, headers=headers)
    else:
        print("Access token retrieval failed.")

    return access_token  # Return the access token

In [6]:
def parse_url():

    url_token = input("Enter the URL: ")
    
    parsed_url = urlparse(url_token)
    query_params = parse_qs(parsed_url.query)
    
    authorization_code = query_params.get("code")[0] if query_params.get("code") else None
    
    return authorization_code

In [24]:
def perform_oauth_authentication():
    token = get_token()

    # Set the date range from last night until today
    today = datetime.now() + timedelta(hours=12)
    yesterday = datetime.now()

    start_time = datetime.combine(yesterday, datetime.min.time())
    end_time = datetime.combine(today, datetime.min.time())

    # Convert start and end time to milliseconds since epoch
    start_timestamp = int(start_time.timestamp() * 1000)
    end_timestamp = int(end_time.timestamp() * 1000)

    # Define the URL for the sleep API
    url = f"https://www.googleapis.com/fitness/v1/users/me/dataset:aggregate"

    # Define the request payload
    request_payload = {
        "aggregateBy": [{
            "dataTypeName": "com.google.sleep.segment"
        }],
        "startTimeMillis": start_timestamp,
        "endTimeMillis": end_timestamp
    }

    # Define the headers with the authorization token
    headers = {
        "Authorization": f"Bearer {token}"
    }

    # Send the POST request to fetch sleep data
    response = requests.post(url, headers=headers, json=request_payload)
    
    

    # Check the response status
    if response.status_code == 200:
        sleep_data = response.json()

        # Dictionary to store sleep stage durations
        sleep_durations = {
            1: timedelta(),  # Awake (during sleep cycle)
            2: timedelta(),  # Sleep
            3: timedelta(),  # Out-of-bed
            4: timedelta(),  # Light sleep
            5: timedelta(),  # Deep sleep
            6: timedelta()   # REM
        }

        # Calculate the duration of each sleep stage
        total_sleep_duration = timedelta()
        for sleep_segment in sleep_data.get("bucket", []):
            for dataset in sleep_segment.get("dataset", []):
                for point in dataset.get("point", []):
                    start_time_nanos = int(point.get("startTimeNanos", 0))
                    end_time_nanos = int(point.get("endTimeNanos", 0))
                    sleep_type = point["value"][0].get("intVal", None)
                    start_time = datetime.fromtimestamp(start_time_nanos / 1e9)
                    end_time = datetime.fromtimestamp(end_time_nanos / 1e9)
                    duration = end_time - start_time

                    # Add the duration to the corresponding sleep stage
                    if sleep_type in sleep_durations:
                        sleep_durations[sleep_type] += duration
                        
                    total_sleep_duration += duration

        # Print the total duration of each sleep stage
        for sleep_type, duration in sleep_durations.items():
            sleep_stage = get_sleep_stage_name(sleep_type)
            print(f"Sleep stage {sleep_stage}: Total duration {duration}")
            
        print(f"Total sleep duration: {total_sleep_duration}")
        print(f"Actual sleep duration: {total_sleep_duration-sleep_durations[1]}")
        print(f"Sleep efficiency: {100*round((total_sleep_duration-sleep_durations[1])/total_sleep_duration, 2)} %")
        
        

    else:
        print("Error fetching sleep data:", response.text)


def get_sleep_stage_name(sleep_type):
    sleep_stages = {
        1: "Awake (during sleep cycle)",
        2: "Sleep",
        3: "Out-of-bed",
        4: "Light sleep",
        5: "Deep sleep",
        6: "REM"
    }
    return sleep_stages.get(sleep_type, "Unknown")



In [25]:
perform_oauth_authentication()

Please visit the following URL to authorize your application:
https://accounts.google.com/o/oauth2/auth?client_id=811981652591-ev7lpah32cjfa45jk4r0m7c8upaeh4q2.apps.googleusercontent.com&redirect_uri=https://www.lmu.de/&scope=https://www.googleapis.com/auth/fitness.sleep.read&response_type=code
Enter the URL: https://www.lmu.de/de/index.html?code=4%2F0AZEOvhV9mQ83Zn9m83CH3l_R8qLb3-pYLeqIgdSWevJPX6GOfYHSdf8RXzboE4iRB_0cBw&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Ffitness.sleep.read
Sleep stage Awake (during sleep cycle): Total duration 0:52:59.959000
Sleep stage Sleep: Total duration 0:00:00
Sleep stage Out-of-bed: Total duration 0:00:00
Sleep stage Light sleep: Total duration 5:38:14.951000
Sleep stage Deep sleep: Total duration 0:41:59.993000
Sleep stage REM: Total duration 0:00:00
Total sleep duration: 7:13:14.903000
Actual sleep duration: 6:20:14.944000
Sleep efficiency: 88.0 %
